# FinanceDataReader 
- https://github.com/FinanceData/FinanceDataReader
- `import financedatareader as fdr`
- `bs4`, `ploty` 의존성 확인하기

## 1.1 FinanceDataReader 라이브러리 실행하기

In [1]:
import FinanceDataReader as fdr
# 삼성전자 종목코드 : 005930
# SK하이닉스 종목코드 : 000660

## 1.2 개별종목의 가격 데이터 불러오기
- `fdr.DataReader()`

In [2]:
df=fdr.DataReader('005930') # 총 6000개 데이터

df2 = fdr.DataReader("000660", start = '2018-04-05', end = '2021-05-31')
df2

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-04-05,82200,83200,81200,82900,3255356,0.031095
2018-04-06,80000,80600,79300,80300,5693281,-0.031363
2018-04-09,80000,81200,79300,80600,3231256,0.003736
2018-04-10,80000,80800,79200,80400,2924054,-0.002481
2018-04-11,82400,82400,81000,81200,3053453,0.009950
...,...,...,...,...,...,...
2021-05-25,121500,123000,120500,123000,3258364,0.029289
2021-05-26,124000,124500,122000,123000,3015522,0.000000
2021-05-27,124500,126000,123000,125500,4539067,0.020325


## 1.3 한국거래소 종목코드 불러오기
- `fdr.StockListing()`

In [3]:
stocks=fdr.StockListing('KRX') # 코스피,코스닥,코넥스 전체
stocks

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,54100,1,400,0.74,53000,54200,52900,11884258,638263378000,322965235955000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,199700,2,-3300,-1.63,197800,203000,197600,1699455,339944006100,145382072290500,728002365,STK
2,207940,KR7207940008,삼성바이오로직스,KOSPI,,1145000,2,-15000,-1.29,1160000,1163000,1140000,49690,57239348000,81494230000000,71174000,STK
3,373220,KR7373220003,LG에너지솔루션,KOSPI,,342500,1,6500,1.93,333000,343500,329000,140721,47375044500,80145000000000,234000000,STK
4,005380,KR7005380001,현대차,KOSPI,,200000,2,-500,-0.25,199500,200500,197700,651492,129806700600,41883238200000,209416191,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,245450,KR7245450002,씨앤에스링크,KONEX,일반기업부,1096,5,-193,-14.97,1100,1100,1096,11,12096,1731636160,1579960,KNX
2869,217320,KR7217320001,썬테크,KONEX,일반기업부,1530,5,-270,-15.00,1600,1600,1530,120,184300,1606500000,1050000,KNX
2870,308700,KR7308700004,테크엔,KONEX,일반기업부,199,0,0,0.00,0,0,0,0,0,1472600000,7400000,KNX
2871,413300,KR7413300005,티엘엔지니어링,KONEX,일반기업부,950,2,-49,-4.90,1009,1009,950,2,1959,1283939250,1351515,KNX


In [4]:
len(stocks)

2873

In [5]:
import time

t1 = time.time()
stocks=fdr.StockListing('KRX') 
t2 = time.time()

print(t2-t1)

7.462245464324951


In [6]:
def StockListing():
    
    import requests
    import pandas as pd
    import json

    try:
        from pandas import json_normalize
    except ImportError:
        from pandas.io.json import json_normalize
    
    data = {'bld': 'dbms/comm/finder/finder_stkisu', 'locale': 'ko_KR', }
    #요청시 넘겨줄 pramaters
    params ={
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
        'locale': 'ko_KR',
        'mktId': 'ALL',
        'share': '1',
        'csvxls_isNo': 'false'
    }
    
    #요청시 헤더 정보
    headers = {
        'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
        'Upgrade-Insecure-Requests': '1', 
        # HTTP Content-Security-Policy (CSP) upgrade-insecure-requests 지시문은 
        # 사이트의 모든 비보안 URL(HTTP를 통해 제공되는 URL)을 
        # 보안 URL(HTTPS를 통해 제공되는)로 대체된 것처럼 처리하도록 사용자 에이전트에 지시
        # [출처] http 헤더 Upgrade-Insecure-Requests|작성자 겨울나무
        'User-Agent': 'Mozilla/5.0'
    }
    
    r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', params, headers=headers)
    print(r)
    
    jo = json.loads(r.text)
    df = json_normalize(jo, 'OutBlock_1')
    df_info = df[['ISU_SRT_CD', 'ISU_ABBRV']]
    
    df_info.columns = ['Symbol', 'Name']
    return df_info

stock_info = StockListing()

<Response [200]>


# 종목에 해당하는 종목코드 찾기

In [7]:
# Example) '바나나'에 해당되는 정보찾기
import pandas as pd

fruits = ['사과', '바나나', '딸기', '복숭아', '수박', '참외', '포도']
price = [1000, 2000, 3000, 4000, 5000, 6000, 7000]
market = ['A마트', 'B마트', 'C마트', 'D마트', 'E마트', 'F마트', 'G마트']
df = pd.DataFrame(
    {"과일":fruits, "가격":price, "파는곳":market})

df.index = range(1,8)

df

,과일,가격,파는곳
1,사과,1000,A마트
2,바나나,2000,B마트
3,딸기,3000,C마트
4,복숭아,4000,D마트
5,수박,5000,E마트
6,참외,6000,F마트
7,포도,7000,G마트


In [8]:
condition = df['과일'] == '바나나'

df[condition]['가격'].values[0]

np.int64(2000)

In [9]:
stock_name = "AJ네트웍스"

stock_info[stock_info['Name'] == stock_name]['Symbol'].values[0]

'095570'

In [10]:
stock_name = "SK하이닉스"

if stock_name in stock_info['Name'].values:
    stock_symbol = stock_info[stock_info['Name']== stock_name]['Symbol'].values[0]
else:
    print("There is no name in KRX")

df = fdr.DataReader(stock_symbol, start = '2021')
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,124500,128000,120500,126000,7995016,0.063291
2021-01-05,124500,132500,124000,130500,7180224,0.035714
2021-01-06,132500,137000,130000,131000,8525749,0.003831
2021-01-07,132500,137500,131500,134500,5981299,0.026718
2021-01-08,136000,138000,132500,138000,8713010,0.026022
...,...,...,...,...,...,...
2025-02-04,192900,194300,190100,191100,5851146,0.001048
2025-02-05,191900,199400,191900,198800,4245118,0.040293
2025-02-06,202000,205000,200500,203500,4032622,0.023642


# StockListing()를 저장하고 불러올 수 있는 코드를 작성

In [11]:
def StockListing():
    
    import requests
    import pandas as pd
    import json

    try:
        from pandas import json_normalize
    except ImportError:
        from pandas.io.json import json_normalize

    #요청시 넘겨줄 pramaters
    params ={
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
        'locale': 'ko_KR',
        'mktId': 'ALL',
        'share': '1',
        'csvxls_isNo': 'false'
    }
    
    #요청시 헤더 정보
    headers = {
        'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
        'Upgrade-Insecure-Requests': '1', 
        'User-Agent': 'Mozilla/5.0'
    }
    
    r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', params, headers=headers)
    print(r)
    
    jo = json.loads(r.text)
    df = json_normalize(jo, 'OutBlock_1')
    df_info = df[['ISU_SRT_CD', 'ISU_ABBRV']]
    
    df_info.columns = ['Symbol', 'Name']
    return df_info

import time

t1 = time.time()

stock_info  = StockListing()

t2 = time.time()

print(t2 - t1)

<Response [200]>
0.9658992290496826


In [12]:
import datetime

today_year = datetime.datetime.today().year
today_month = datetime.datetime.today().month
today_day = datetime.datetime.today().day
print(today_year, today_month, today_day)

2025 2 10


In [13]:
krx_df = StockListing()
krx_df

<Response [200]>


,Symbol,Name
0,098120,마이크로컨텍솔
1,009520,포스코엠텍
2,095570,AJ네트웍스
3,006840,AK홀딩스
4,282330,BGF리테일
...,...,...
2868,000545,흥국화재우
2869,000540,흥국화재
2870,003280,흥아해운
2871,037440,희림


In [14]:
# krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))

In [15]:
import os

os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))

False

In [16]:
if os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)):
    krx_df = pd.read_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day), index_col = 0)
else:
    krx_df = StockListing()
    krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))


<Response [200]>


In [17]:
krx_df

,Symbol,Name
0,098120,마이크로컨텍솔
1,009520,포스코엠텍
2,095570,AJ네트웍스
3,006840,AK홀딩스
4,282330,BGF리테일
...,...,...
2868,000545,흥국화재우
2869,000540,흥국화재
2870,003280,흥아해운
2871,037440,희림


# 함수 만들기_(stockDataReader_fn)

In [18]:
def stockDataReader_fn(stock_name, start_date = None, end_date =None): 

    import FinanceDataReader as fdr
    import datetime
    import os
    import pandas as pd

    def StockListing():
        
        import requests
        import json

        try:
            from pandas import json_normalize
        except ImportError:
            from pandas.io.json import json_normalize

        #요청시 넘겨줄 pramaters
        params ={
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
            'locale': 'ko_KR',
            'mktId': 'ALL',
            'share': '1',
            'csvxls_isNo': 'false'
        }
        
        #요청시 헤더 정보
        headers = {
            'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
            'Upgrade-Insecure-Requests': '1', 
            'User-Agent': 'Mozilla/5.0'
        }
        
        r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', params, headers=headers)
        print(r)
        
        jo = json.loads(r.text)
        df = json_normalize(jo, 'OutBlock_1')
        df_info = df[['ISU_SRT_CD', 'ISU_ABBRV']]
        
        df_info.columns = ['Symbol', 'Name']
        return df_info
    #1. 저장 파일 형식에 필요한 Data 불러오기
    today_year = datetime.datetime.today().year
    today_month = datetime.datetime.today().month
    today_day = datetime.datetime.today().day
    
    # 2. KRX 종목 DataFrame 존재 여부 확인
    if os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)):
        krx_df = pd.read_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day), index_col = 0)
    else:
        krx_df = StockListing()
        krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))
    
    # 3. 종목 코드 찾기
    if stock_name in krx_df['Name'].values:
        symbol = krx_df[krx_df['Name'] == stock_name]['Symbol'].values[0]
    else:
        print("There is no name in KRX")
        raise "Threr is no name in KRX" # 함수가 중간에 멈추고 -> error 발생
    
    # 찾은 심볼을 가지고 -> 시작, 끝 날자의 상장 주식 정보를 반환함 !
    stock_df = fdr.DataReader(symbol, start_date, end_date)
    
    return stock_df





In [19]:
stock_name = "LG전자"
start_date = '2020'
end_date = None
df = stockDataReader_fn(stock_name = stock_name, start_date = start_date, end_date = end_date)
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,71800,72100,71000,71000,284758,-0.015257
2020-01-03,71400,71900,70800,71100,369493,0.001408
2020-01-06,70300,70500,69600,69900,337026,-0.016878
2020-01-07,69800,70600,69800,70300,265311,0.005722
2020-01-08,70300,70300,67000,67500,937438,-0.039829
...,...,...,...,...,...,...
2025-02-04,79600,79700,78100,78400,629803,0.002558
2025-02-05,79100,80300,78900,79900,434946,0.019133
2025-02-06,80400,80600,79500,80600,401172,0.008761
